# regression hw in week 13

**info**

SuJinhua 2017201620

**DIY regression function**

In [186]:
def linear_regression(x,y):
    x = np.hstack((np.ones((x.shape[0],1)),x))
    beta = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(y))
    return beta
def logit_regression(x,y,t=100,init_beta =False,beta = None,method = 'newton',lr = 0.001):
    x = np.hstack((np.ones((x.shape[0],1)),x))
    if init_beta == False:
        beta = np.zeros((x.shape[1],1))
    def p1(x,y,beta):
        return 1/(1+np.exp(-1 * x.dot(beta)))
    def derivate_1(x,y,beta):
        return -1 * x.T.dot((y-p1(x,y,beta).T).T)
    def derivate_2(x,y,beta):
        tmp = p1(x,y,beta)
        tmp = tmp*(1-tmp)
        return x.T.dot(x*tmp)
    if method == 'newton':
        for i in range(t):
            beta = beta - np.linalg.inv(derivate_2(x,y,beta)).dot(derivate_1(x,y,beta))
    elif method == 'simple':
        for i in range(t):
            beta = beta - lr * derivate_1(x,y,beta)
    return beta

## 10.5

- For group data, we use linear regression to estimate the logistic regression.
- By some data operations, we can get the raw ungrouped data

In [230]:
import pandas as pd
import numpy as np
from sklearn import linear_model
df1 = pd.read_excel('table10-11.xlsx')
df2 = pd.read_excel('table10-12.xlsx')
df1.head()

,num,x1,x2,ni,y=1,y=0,pi
0,1,22,0,3,0,3,0.12500
1,2,22,1,11,3,8,0.29167
2,3,22,2,389,146,243,0.37564
3,4,22,3,83,26,57,0.31548
4,5,37,0,4,3,1,0.70000


In [79]:
df1['p_'] = np.log(df1['pi']/(1 - df1['pi']))
y = df1['p_'].to_numpy()
x = df1[['x1','x2']].to_numpy()

In [241]:
lr = linear_model.LinearRegression()
lr.fit(x,y)
print('cont:',lr.intercept_)
print('ceof:',lr.coef_)

cont: -0.1444940115598669
ceof: [-0.00631765 -0.13643885]


In [42]:
linear_regression(x,y)

array([-0.14449401, -0.00631765, -0.13643885])

### subproblem 1

Since the grouped data has an estimate of probability, the logistic regression can be converted to linear regression. The coefficients are all negative, that means that with age and education increase, citizen may be less afraid of the stranger. But the influence of age increase is implicit, eduction may be a good factor to explain the explantory variable.

In [242]:
x2,y2 = df_new[['x1_d','x2_d']].to_numpy(),df_new['y'].to_numpy()
logi_r = linear_model.LogisticRegression(solver='newton-cg')
logi_r.fit(x2,y2)
print('intercept:',logi_r.intercept_)
print('ceof:',logi_r.coef_)

intercept: [0.11996409]
ceof: [[ 0.00099379 -0.30977755]]


In [246]:
def generate_seperate_data(df):
    x1_list,x2_list,y_list = [],[],[]
    for i in range(len(df)):
        n = df.loc[i,'ni']
        n_1 = df.loc[i,'y=1']
        x1_list += [df.loc[i,'x1']] * n
        x2_list += [df.loc[i,'x2']] * n
        y_list += [1] * n_1 + [0] * (n - n_1)
    return pd.DataFrame({'x1_d':x1_list,'x2_d':x2_list,'y':y_list})
df_new = generate_seperate_data(df1)
df_new.head()

,x1_d,x2_d,y
0,22,0,0
1,22,0,0
2,22,0,0
3,22,1,1
4,22,1,1


In [245]:
logit_regression(df_new[['x1_d','x2_d']].to_numpy(),df_new['y'].to_numpy(),method = 'newton',lr =0.1,t = 200)

array([[ 0.12862471],
       [ 0.00097967],
       [-0.31368478]])

### subproblem 2

As for the ungrouped data, we use newton method to iterate the beta. To check my DIY logistic regression correctness, I use sklearn regression package to check the result. This time the coefficient of the age(x1) is positive but really small, while the coefficient of education(x2) becomes more influent.

## 10.6

In [22]:
df2.head()

,num,h,y
0,1,1.50,0
1,2,1.52,0
2,3,1.54,0
3,4,1.56,0
4,5,1.58,1


In [247]:
x3 = df2[['h']].to_numpy()
y3 = df2['y'].to_numpy()
logi_r2 = linear_model.LogisticRegression(solver='lbfgs')
logi_r2.fit(x3,y3)
print('intercept:',logi_r2.intercept_)
print('ceof:',logi_r2.coef_)

intercept: [-1.88627383]
ceof: [[0.80832952]]


In [228]:
logi_r2.predict(x3)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [249]:
beta = logit_regression(x3,y3)
beta

array([[-14.59176501],
       [  7.98143472]])

In [239]:
def p1(x,y,beta):
    return 1/(1+np.exp(-1 * x.dot(beta)))
my_pred = p1(np.hstack((np.ones((len(x3),1)),x3)),y3,beta).squeeze()>0.5
my_pred = my_pred.astype(np.int16) 
my_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1], dtype=int16)

In [240]:
y3

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1])

### conclusion

I use my DIY logistic regression to predict train data and find one feature could not fully explain the result. There are ignored variables, such as the soomth of the ground and the shape of the glass. I use the sklearn package to fit the logistic regression, but it gives a different result. However the beta0 / beta1 = 1.8, which means the 1.8 meter is the therohold of the glass.